In [2]:
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [3]:
import tensorflow as tf
import os
from tqdm import tqdm
import urllib.request as req

In [11]:
SOURCE_URL = "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip"

In [12]:
DESTINATION = "data"
os.makedirs(DESTINATION, exist_ok=True)
data_file="data.zip"
DESTINATION_ZIP_PATH = os.path.join(DESTINATION,data_file)

In [13]:
DESTINATION_ZIP_PATH

'data/data.zip'

In [14]:
pwd

'/home/hari/deep_learning/CNN-classifier-fsdsf21/research_env'

In [16]:
filename,headers = tqdm(req.urlretrieve(SOURCE_URL,DESTINATION_ZIP_PATH))

100%|██████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 23967.45it/s]


In [17]:
filename

'data/data.zip'

In [20]:
print(headers)

Content-Type: application/octet-stream
Accept-Ranges: bytes
Server: Windows-Azure-Blob/1.0 Microsoft-HTTPAPI/2.0
x-ms-blob-content-md5: 4TekUHNw2UJGm20meiTqBA==
Last-Modified: Mon, 09 May 2022 20:42:07 GMT
ETag: "0x8DA31FC621E01C4"
Content-Length: 824887076
Date: Sat, 25 Feb 2023 08:04:23 GMT
Connection: close


